In [1]:
import argparse
import sys
import torch

In [2]:
def main():
    # Parse command line arguments
    parser = argparse.ArgumentParser(description='Translate using a pre-trained model')
    parser.add_argument('--model', help='a model previously trained with train.py')
    parser.add_argument('--batch_size', type=int, default=50, help='the batch size (defaults to 50)')
    parser.add_argument('--beam_size', type=int, default=12, help='the beam size (defaults to 12, 0 for greedy search)')
    parser.add_argument('--encoding', default='utf-8', help='the character encoding for input/output (defaults to utf-8)')
    parser.add_argument('-i', '--input', default=sys.stdin.fileno(), help='the input file (defaults to stdin)')
    parser.add_argument('-o', '--output', default=sys.stdout.fileno(), help='the output file (defaults to stdout)')
    args = parser.parse_args(args=[])

    args.batch_size=128
    args.model="model/arm32_to_x86.it400.src2trg.pth"
    args.input="data/arm32_corpus/arm32"
    args.output="data/output_translate"

    # Load model
    translator = torch.load(args.model)

    # Translate sentences
    end = False
    fin = open(args.input, encoding=args.encoding, errors='surrogateescape')
    fout = open(args.output, mode='w', encoding=args.encoding, errors='surrogateescape')
    while not end:
        batch = []
        while len(batch) < args.batch_size and not end:
            line = fin.readline()
            if not line:
                end = True
            else:
                batch.append(line)
        if args.beam_size <= 0 and len(batch) > 0:
            for translation in translator.greedy(batch, train=False):
                print(translation, file=fout)
        elif len(batch) > 0:
            for translation in translator.beam_search(batch, train=False, beam_size=args.beam_size):
                print(translation, file=fout)
        fout.flush()
    fin.close()
    fout.close()

if __name__ == '__main__':
    main()    

/home/mhu20/Downloads/UnsuperBinTrans/code/undreamt/undreamt/translator.py:59: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  varids = self.device(Variable(torch.LongTensor(ids), requires_grad=False, volatile=not train))
/home/mhu20/Downloads/UnsuperBinTrans/code/undreamt/undreamt/attention.py:45: UserWarning: masked_fill_ received a mask with dtype torch.uint8, this behavior is now deprecated,please use a mask with dtype torch.bool instead. (Triggered internally at /home/conda/feedstock_root/build_artifacts/pytorch-recipe_1673730874951/work/aten/src/ATen/native/cuda/Indexing.cu:1435.)
  align.data.masked_fill_(mask, -float('inf'))


: 